# Import packages

In [56]:
import pandas as pd
import re

import sys
sys.path.append('../')
from aqc_utils.gaussian_log_extractor import *
from aqc_utils.molecule import molecule
from aqc_utils.slurm_manager import slurm_manager
from aqc_utils.helper_classes import slurm_status

import logging
logging.basicConfig(level=logging.INFO)

# Objectif :
definir une fonction qui permet de generer un dataframe ou d'updater un .csv avec les molecules calculées et toutes leurs caracteristiques

## First Step
Recupérer les datas d'une molécule

In [57]:
sm=slurm_manager(user='jschleinitz', host='hermite.chimie.ens.fr')
sm.connect()

INFO:aqc_utils.slurm_manager:Creating connection to hermite.chimie.ens.fr as jschleinitz
INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.2p1)
INFO:paramiko.transport:Authentication (password) successful!
INFO:aqc_utils.slurm_manager:Connected to hermite.chimie.ens.fr as jschleinitz is True.


In [58]:
sm.get_job_stats(split_by_can=True)

status,created,submitted,uploaded
can,,,
C(Oc1ccccc1)Oc1ccccc1,1,0,0
C=CC[Si](Oc1ccc2c(c1)cccc2)(C)C,1,0,0
CC(=O)Oc1c2ccccc2cc2c1cccc2,1,0,0
CC(=O)Oc1ccc(cc1)C(=O)C,1,0,0
CC(=O)Oc1ccc2c(c1)ccc(c2)C(=O)C,1,0,0
...,...,...,...
OCCOc1ccc(cc1)C,1,0,0
OCCOc1ccccc1,1,0,0
OCCc1ccc(cc1)OC,0,0,1


# It is very important here to update with a suited tag name the molecules !


In [59]:
sm.upload_done_molecules_to_db(tags="JS_substrate")

INFO:aqc_utils.slurm_manager:There are no jobs in done status. Exitting.


In [60]:
for job in sm.get_jobs():
    if str(sm.get_jobs()[job].status) == 'slurm_status.uploaded':
        job_to_change = sm.get_jobs()[job].base_name
        with sm.connection.cd("/home/jschleinitz/gaussian/"):
            try:
                sm.connection.run(f"mv {job_to_change}.chk ../gaussian_uploaded")
                sm.connection.run(f"mv {job_to_change}.out ../gaussian_uploaded")
                sm.connection.run(f"mv {job_to_change}.log ../gaussian_uploaded")
                sm.connection.run(f"mv {job_to_change}.com ../gaussian_uploaded")
                sm.connection.run(f"mv {job_to_change}.sh ../gaussian_uploaded")
                break
            except:
                pass

mv: cannot stat 'C11H10O_b7a8_conf_0.chk': No such file or directory
mv: cannot stat 'C14H18OSi_95fa_conf_0.chk': No such file or directory
mv: cannot stat 'C11H10O2_d7c7_conf_0.chk': No such file or directory
mv: cannot stat 'C11H10O_9eb4_conf_0.chk': No such file or directory
